In [1]:
import recordlinkage
from Annotation_helper import *
import numpy as np

In [2]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

In [3]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(permissions, transactions)

compare_cl = recordlinkage.Compare()

compare_cl.exact('SlaafGender', 'SlaafGender', label='SlaafGender')
compare_cl.string('SlaafNaamNieuw', 'SlaafNaamNieuw', label='SlaafNaamNieuw', method='levenshtein')
compare_cl.exact('BezitterGender', 'KoperGender', label='BezitterGender')
compare_cl.string('BezitterVoornaam', 'KoperVoornaam', label='BezitterVoornaam', method='levenshtein')
compare_cl.string('BezitterBeroep', 'KoperBeroep', label='BezitterBeroep', method='levenshtein')
compare_cl.string('BezitterAchternaam', 'KoperAchternaam', label='BezitterAchternaam', method='levenshtein')
n_batches = 100
split_perm = np.array_split(permissions, n_batches)

In [5]:
for i, p in enumerate(split_perm):
    print(f"[{i}/{n_batches}]")
    candidate_links = indexer.index(p, transactions)
    features = compare_cl.compute(candidate_links, p, transactions)
    features = features.reset_index()
    features = features.rename(columns={"level_0": "permission_indx", "level_1": "transaction_indx"})
    features.to_csv(f"Distances/distances_{i}.csv", index=False)
    clear_output(wait=True)

[99/100]
